#  Code to filter transparent nederland data and produce a gephi file

In [4]:
## All imports go here
import pandas as pd
from dateutil.parser import parse
import re
import numpy as np

#def delete_after_comma(x): # now done in filter-organizations.py
#    ind = x.find(",")
#    if ind > 0:
#        return x[:x.find(",")]
#    else: return x

##  Step 1. Filter namen

In [ ]:
#

##  Step 2. Merge files to have organizations, political parties and people in one nodes file

In [5]:
df = pd.read_csv("./data/details-organisationsextracted.csv",index_col=None)
df = df.dropna(subset=["organisatie"])

#df["organisatie"] = df["organisatie"].apply(lambda x: delete_after_comma(x)) # now done in filter-organizations.py
df2 = df.copy()
nodes1 = df2[['b1-nummer',"organisatie"]]
nodes1.columns = ['Type_x',"Label"]
nodes1["Type_x"] = 'Org'

df = pd.read_csv("./data/namen.csv",index_col=None)
nodes2 = df[["b1-nummer","partij(en)/fractie(s)"]]
nodes2.columns = ["Type","Label"]
nodes2["Type"] = 'Party'

nodes = pd.merge(nodes1,nodes2,on="Label",how="outer")
nodes.loc[nodes["Type"] != "Party","Type"]  = nodes.loc[nodes["Type"] != "Party","Type_x"]
del nodes["Type_x"]

nodes["Id"] = [i for i in range(nodes.shape[0])]

df = pd.read_csv("./data/namen.csv",index_col=None)
df["Naam"] = df[[ 'voorletters', 'voorna(a)m(en)','achternaam']].apply(lambda x: "{} {} {}".format(*x),axis=1)

nodes3 = df[["b1-nummer","Naam","functie"]]
nodes3.columns = ["b1-nummer","Label","Type_x"]
nodes3["Type_x"] = 'Person'

nodes = pd.merge(nodes,nodes3,on="Label",how="outer")
nodes = nodes.drop_duplicates(subset=["Label"])

nodes.loc[nodes["Type_x"] == "Person","Type"]  = 'Person'
del nodes["Type_x"]
nodes["Id"] = [i for i in range(nodes.shape[0])]



b1_to_id = nodes.copy()

b1_to_id = b1_to_id.dropna(subset=["b1-nummer"])
b1_to_id = b1_to_id[["b1-nummer","Id"]]
b1_to_id["b1-nummer"] = b1_to_id["b1-nummer"].apply(lambda x: int(x))

label_to_id = nodes.loc[nodes["Type"] != "Person",:].copy()
label_to_id = label_to_id.dropna(subset=["Label"])
label_to_id = label_to_id[["Label","Id"]].sort()
label_to_id.columns = ["Target_label","Id"]

del nodes["b1-nummer"]
nodes.to_csv("./data/nodes_bipartite.csv",index=None,sep="\t")
print(nodes.shape)

/home/javier/.local/lib/python3.4/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/javier/.local/lib/python3.4/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/javier/.local/lib/python3.4/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/index

(16375, 3)


/home/javier/.local/lib/python3.4/site-packages/ipykernel/__main__.py:49: FutureWarning: sort(....) is deprecated, use sort_index(.....)


##  Step 3. Filter

In [4]:
## merge so we have an id, and find timestamp

def separate_datum_org(date):
    if isinstance(date,float): return [np.NaN,np.NaN]
    
    all_slashes = [m.start() for m in re.finditer('/', date)]
    if len(all_slashes) == 0: return [np.NaN,np.NaN]
    
    date = date.replace("00","01")    
    middle_slash = all_slashes[int(len(all_slashes)/2)]
    
    date_in = date[:middle_slash]
    date_out = date[middle_slash+1:]
    if len(date_in) < 2 or len(date_in) < 2: return [np.NaN,np.NaN]
    try: return parse(date_in,fuzzy=True).date(), parse(date_out,fuzzy=True).date()
    except: return [np.NaN,np.NaN]

def separate_datum_peop(date):
    if isinstance(date,float): return np.NaN
    return parse(date).date()

df = pd.read_csv("./data/details-organisationsextracted.csv",index_col=None)
df = df.dropna(subset=["organisatie"])
#df["organisatie"] = df["organisatie"].apply(lambda x: delete_after_comma(x))  # now done in filter-organizations.py
dates = df["datum"].apply(lambda x: separate_datum_org(x))
date_in, date_out = zip(*dates)
df["date_in"] = date_in
df["date_out"] = date_out
print(df.columns)
df = df[["b1-nummer",'organisatie','date_in', 'date_out','rubriek']]

df.columns = ['b1-nummer', 'Target_label', 'date_in', 'date_out', 'rubriek']

df2 = pd.read_csv("./data/namen.csv",index_col=None,dtype=object)
df2["date_in"] = df2["begin periode"].apply(lambda x: separate_datum_peop(x))
df2["date_out"] = df2["einde periode"].apply(lambda x: separate_datum_peop(x))
df2 = df2[["b1-nummer",'partij(en)/fractie(s)','date_in', 'date_out','functie']]
df2.columns = ["b1-nummer",'Target_label','date_in', 'date_out','rubriek']
df2['rubriek'] = "-9"

df = pd.concat([df,df2])
df["b1-nummer"] = df["b1-nummer"].apply(lambda x: int(x))
df = df.dropna(subset=["Target_label"])

print(label_to_id.shape)
print(b1_to_id.shape)

edges = pd.merge(df,b1_to_id,on="b1-nummer",how="inner")
del edges["b1-nummer"]
edges.columns = ['Target_label', 'date_in', 'date_out', 'rubriek', 'Source']
edges = pd.merge(edges,label_to_id,on = "Target_label",how="inner")

edges = edges.loc[edges["Target_label"] != "",:]
edges = edges.drop_duplicates()
del edges["Target_label"]
edges.columns = ["date_in","date_out","rubriek","Source","Target"]
edges = edges.loc[:,["Source","Target","date_in","date_out","rubriek"]]
edges.to_csv("./data/edges_bipartite.csv",index=None,sep="\t")

Index(['b1-nummer', 'rubriek', 'organisatie', 'type', 'waarde', 'datum',
       'toelichting', 'date_in', 'date_out'],
      dtype='object')
(15457, 2)
(917, 2)


In [ ]:
import numpy as np
np.array?

## Filter all organizations with degree 1

In [6]:
## merge so we have an id, and find timestamp

def separate_datum_org(date):
    if isinstance(date,float): return [np.NaN,np.NaN]
    
    all_slashes = [m.start() for m in re.finditer('/', date)]
    if len(all_slashes) == 0: return [np.NaN,np.NaN]
    
    date = date.replace("00","01")    
    middle_slash = all_slashes[int(len(all_slashes)/2)]
    
    date_in = date[:middle_slash]
    date_out = date[middle_slash+1:]
    if len(date_in) < 2 or len(date_in) < 2: return [np.NaN,np.NaN]
    try: return parse(date_in,fuzzy=True).date(), parse(date_out,fuzzy=True).date()
    except: return [np.NaN,np.NaN]

def separate_datum_peop(date):
    if isinstance(date,float): return np.NaN
    return parse(date).date()

df = pd.read_csv("./data/details-organisationsextracted.csv",index_col=None)
df = df.dropna(subset=["organisatie"])
#df["organisatie"] = df["organisatie"].apply(lambda x: delete_after_comma(x))  # now done in filter-organizations.py
df2 = df.groupby("organisatie").count()
df2 = df2.loc[df2["b1-nummer"]>1,:]
df = df.loc[df2["organisatie"].isin(df2.index),:]
df
              
break


dates = df["datum"].apply(lambda x: separate_datum_org(x))
date_in, date_out = zip(*dates)
df["date_in"] = date_in
df["date_out"] = date_out
print(df.columns)
df = df[["b1-nummer",'organisatie','date_in', 'date_out','rubriek']]

df.columns = ['b1-nummer', 'Target_label', 'date_in', 'date_out', 'rubriek']

df2 = pd.read_csv("./data/namen.csv",index_col=None,dtype=object)
df2["date_in"] = df2["begin periode"].apply(lambda x: separate_datum_peop(x))
df2["date_out"] = df2["einde periode"].apply(lambda x: separate_datum_peop(x))
df2 = df2[["b1-nummer",'partij(en)/fractie(s)','date_in', 'date_out','functie']]
df2.columns = ["b1-nummer",'Target_label','date_in', 'date_out','rubriek']
df2['rubriek'] = "-9"

df = pd.concat([df,df2])
df["b1-nummer"] = df["b1-nummer"].apply(lambda x: int(x))
df = df.dropna(subset=["Target_label"])

print(label_to_id.shape)
print(b1_to_id.shape)

edges = pd.merge(df,b1_to_id,on="b1-nummer",how="inner")
del edges["b1-nummer"]
edges.columns = ['Target_label', 'date_in', 'date_out', 'rubriek', 'Source']
edges = pd.merge(edges,label_to_id,on = "Target_label",how="inner")

edges = edges.loc[edges["Target_label"] != "",:]
edges = edges.drop_duplicates()
del edges["Target_label"]
edges.columns = ["date_in","date_out","rubriek","Source","Target"]
edges = edges.loc[:,["Source","Target","date_in","date_out","rubriek"]]
edges.to_csv("./data/edges_bipartite_organizations_filtered.csv",index=None,sep="\t")

KeyError: 'organisatie'